In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
pip install --user datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8

In [1]:
import re

In [2]:
from datasets import load_dataset
import json

ds = load_dataset("BAAI/TACO", split="train")

The repository for BAAI/TACO contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/BAAI/TACO.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/25443 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
filtered_ds = ds.filter(lambda example: example['solutions'] != '[]')

Filter:   0%|          | 0/25443 [00:00<?, ? examples/s]

In [4]:
type(filtered_ds)

datasets.arrow_dataset.Dataset

In [5]:
filtered_ds.num_rows

19493

In [6]:
def convert_to_function(code):
    lines = code.split('\n')
    function_code = ""
    indentation_level = 0

    for line in lines:

        # Count leading tabs to determine indentation level
        indentation = line.count('\t')

        # Add indentation spaces
        indentation_spaces = "    " * indentation

        # Remove tabs from the line
        line = line.replace('\t', '')

        # Add the line with proper indentation
        function_code += indentation_spaces + line + "\n"

    return function_code

# Given code snippet
code_snippet = """t = int(input())\nfor _ in range(t):\n\tn = int(input())\n\tp = list(map(int, input().split()))\n\tans = []\n\tp1 = [-1] * (n + 1)\n\tfor i in range(n):\n\t\tp1[p[i]] = i\n\ti = n\n\twhile i:\n\t\twhile i > 0 and p1[i] == -1:\n\t\t\ti -= 1\n\t\telse:\n\t\t\tif i:\n\t\t\t\tk = 0\n\t\t\t\tfor j in range(p1[i], n):\n\t\t\t\t\tans.append(p[j])\n\t\t\t\t\tp1[p[j]] = -1\n\t\t\t\t\tk += 1\n\t\t\t\tn -= k\n\t\t\t\ti -= 1\n\t\t\telse:\n\t\t\t\tbreak\n\tprint(*ans)\n"""

# Convert the code snippet to a function
function_code = convert_to_function(code_snippet)

print(function_code)


t = int(input())
for _ in range(t):
    n = int(input())
    p = list(map(int, input().split()))
    ans = []
    p1 = [-1] * (n + 1)
    for i in range(n):
        p1[p[i]] = i
    i = n
    while i:
        while i > 0 and p1[i] == -1:
            i -= 1
        else:
            if i:
                k = 0
                for j in range(p1[i], n):
                    ans.append(p[j])
                    p1[p[j]] = -1
                    k += 1
                n -= k
                i -= 1
            else:
                break
    print(*ans)




In [7]:
import json
import random
import ast

def convert_example_to_json(example):
    # Convert solutions and tags from strings to lists
    solutions = example['solutions'].split("\n")
    tags = example['skill_types'].split(",")

    # Skip data points with empty solutions
    if not solutions:
        return None

    # Construct instruction string
    instruction = "Question: {}. Difficulty: {}. Skills: {}. Expected Time Complexity: {}. Expected Auxiliary Space: {}.".format(
        example['question'], example['difficulty'], ', '.join(tags),
        example['Expected Time Complexity'], example['Expected Auxiliary Space']
    )

    # Select a random solution from the list of solutions
    selected_solution = random.choice(solutions)
    # Construct JSON data entry
    data_entry = {
        "instruction": instruction,
        "input": convert_to_function(example['starter_code']),
        "output": convert_to_function(random.choice(ast.literal_eval(example['solutions'])))
    }
    return data_entry

# Convert single example to JSON format
sample = random.choice(filtered_ds)

example_json = convert_example_to_json(sample)

# Print JSON data for the single example
if example_json:
    json_data = json.dumps(example_json, indent=4)
    print(json_data)
else:
    print("Example has no solutions.")


{
    "instruction": "Question: You are given two arrays A and B of length N. Let S be the set of integers from 1 to N. Can you find the maximum possible value of (A_{i1}+A_{i2}+...+A_{ik})^{2}+(B_{i1}+B_{i2}+...+B_{ik})^{2} where {i1,i2...ik} is a non-empty subset of S?\n\nInput Format \n\nThe first line contains a single integer T, denoting the number of test cases. \n\nT testcases follow, each test case given in following format.  \n\nN  \nA1 A2 ... AN  \nB1 B2 ... BN  \n\nOutput Format \n\nFor each test case, output the maximum possible value in one line.  \n\nConstraints \n\n1 <= T <= 10 \n\n1 <= N <= 1000 \n\n-10^{6} <= A_{i}, B_{i} <= 10^{6}  \n\nSample Input  \n\n1  \n2  \n-1 5  \n4 -5  \n\nSample Output  \n\n50\n\nExplanation \n\nAll possible non-empty subsets for N = 2 of S = {1,2} are {1}, {2} and {1,2}. The maximum possible values of the above equation now are \n\n(-1)^{2} + (4)^{2} = 17  \n(5)^{2} + (-5)^{2} = 50  \n(-1 + 5)^{2} + (4 - 5)^{2} = 17\n\nhence 50. \n\nTimelimi

In [8]:
json_data = []
for example in filtered_ds:
    json_entry = convert_example_to_json(example)
    if json_entry is not None:
        json_data.append(json_entry)


In [9]:
json_data[15001]

{'instruction': 'Question: Write the function `resistor_parallel` that receive an undefined number of resistances parallel resistors and return the total resistance.\n\nYou can assume that there will be no 0 as parameter.  \nAlso there will be at least 2 arguments.\n\nFormula:  \n`total = 1 / (1/r1 + 1/r2 + .. + 1/rn)`\n\nExamples:  \n`resistor_parallel(20, 20)` will return `10.0`  \n`resistor_parallel(20, 20, 40)` will return `8.0`. Difficulty: EASY. Skills: []. Expected Time Complexity: None. Expected Auxiliary Space: None.',
 'input': 'def resistor_parallel(*resistors):\n    \n',
 'output': 'from functools import reduce\n\ndef resistor_parallel(*res):\n    return 1 / reduce(lambda x, y: x + y, [1 / x for x in res])\n\n'}

In [10]:
len(json_data)

19493

In [12]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
file_path = '/content/drive/My Drive/output.json'


In [14]:
with open(file_path, 'w') as f:
    json.dump(json_data, f, indent=4)

print(f'JSON data saved to {file_path}')


JSON data saved to /content/drive/My Drive/output.json


In [ ]:
output_file = 'formatted_dataset.json'
with open(output_file, 'w') as f:
    json.dump(json_data, f, indent=4)

print(f"Saved {len(json_data)} entries to {output_file}")

Saved 19493 entries to formatted_dataset.json


In [ ]:
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.DataFrame(json_data))

# Push to Hugging Face Hub


In [ ]:
dataset.push_to_hub("codeT5p-19k")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/subbu264/subbu264-hi/commit/01f25d0d62c91c95961e9cb8459828543a5f24fa', commit_message='Upload dataset', commit_description='', oid='01f25d0d62c91c95961e9cb8459828543a5f24fa', pr_url=None, pr_revision=None, pr_num=None)